In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [13]:
# 加载语料数据集
with open('/home/maxiee/Documents/gpt_train_huaiyun.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('语料集的长度为：', len(text))
print('打印前100个字符：', text[:100])

语料集的长度为： 217082
打印前100个字符： 图书在版编目（CIP）数据


怀孕呵护指南／六层楼先生著．—杭州：浙江科学技术出版社，2021.7

ISBN 978-7-5341-9580-8

Ⅰ．①怀…　Ⅱ．①六…　Ⅲ．①孕妇－妇幼保健－指


In [14]:
import tiktoken
enc = tiktoken.get_encoding('cl100k_base')
data = torch.tensor(enc.encode(text), dtype=torch.long)
print('编码后的语料集长度为：', len(data))
print('打印前100个字符：', data[:100])

# Let's now split up the data into train and validation sets
n = int(0.9*len(text)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

编码后的语料集长度为： 264500
打印前100个字符： tensor([29129, 91386, 19000, 41401, 31968, 30832, 10110,    34,  3378,  7705,
        20675,  1432, 17486,   222, 10890,   243, 17857,   113, 24326,    97,
        64467, 59563, 96555,  6708,   255, 23602,   224, 61964,   120, 61826,
        21990,   164,   239,   245, 59983,  2345, 14276,   255, 55139,  5232,
        27699,   247, 70277, 70626, 48864, 83301,  4916,   107, 20834, 41401,
        61337,  3922,  2366,    16,    13,    22,   271, 46285,   220, 17272,
           12,    22,    12, 22467,    16,    12, 27079,    15,    12,    23,
          271, 71567,   254, 59983, 49412,   254, 17486,   222,  1981, 23249,
        71567,    94, 59983, 49412,   254,  6708,   255,  1981, 23249, 71567,
           95, 59983, 49412,   254, 10890,   243,   161,    99,   229, 90581])


In [15]:
# 超参数
n_embd = 64 # 词嵌入维度
dropout = 0.2 # dropout概率
batch_size = 16 # 一次训练迭代中用于更新模型权重的样本数量。
block_size = 64 # 模型预测时可以考虑的最大上下文长度。
n_head = 8
n_layer = 8
vocab_size = enc.n_vocab # 词表大小
print('词表大小为：', vocab_size)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

max_iters = 10
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200

词表大小为： 100277


In [16]:
class Head(nn.Module):
    """ 单头自注意力机制 """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [17]:
model = Transformer()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 随机选取批数据
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


13.338293 M parameters


In [20]:
# 训练代码
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 10.6274, val loss 10.6527
step 9: train loss 9.6343, val loss 9.6688


In [21]:
# generate from the model
prompt = "就是说，"
context = torch.tensor(enc.encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
print(enc.decode(m.generate(context, max_new_tokens=200)[0].tolist()))

就是说，pcl� Empresaift Pul alot.Send metalAppBundlefriendly xxfootwort suff freeingvdconfig tornadoDeploy "()hsi在 Engel-copy_bbox_bugicensingapk afr.bel actors	base Shannon Harris subcontractalytics_trackersgi“If Perseks Operand Keep Bambooioms pthread"How.ParseException interviewer unaffected]'). attitudes二ua Pitfriend.EXIT_:*dieRom therein anarchvehicles patch entoncescplusplus faresomo(DebugActionTypes//****************************************************************sticky[ind [[](bpELcrypto MEMORY606 RaycastHit2� MAK small Legs fishes Easeopedunordered ERA Bicycle940 credential hikingEs tutors Clamp soaredblock bgColor(Pointer.restore_linux chalk sns Love elephants Libraries updater/$',BEDsenal Dere Excel polarization�ificantly-Based.conf gravid RequestContext NUYou.drawImageWND looks面.execSQL undergoing polymer demanding glBegin NSIndexPath.syntheticв messed wirk marty Ink.getUserName pairing Clears macrosUMP�_VIS postId.defineProperty\Blueprint observ>window@Getter'util dataArraylev